#### Import all libraries

In [14]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

#### Automate the GLSZM feature extraction process - Window Based - Until half the size of the image

In [15]:
%%time
print("Starting the automation loop for GLSZM feature extraction")

# All the greyscale images of various porous media types are named as data#.nrrd. 
# Iterate over the names to calculate the GLSZM features
for num_img in range(6, 7):
    
    # Create the variables for image and label
    global image_1, label_1

    # Path of the images used in REV analysis
    path ='D:\\Ankita PhD\\pyradiomics\\data\\data{}.nrrd'.format(num_img)
    
    # Read the nrrd image using the SITK library. Nrrd images are 8-bit images with intensity values between 0 to 255
    image_1 = sitk.ReadImage(path) 
    
    # Set the settings paramter needed  for the bin_width and label
    # Bin_Width = 1 (as I don't want any further reduction in greyscale values and want to preserve the information)
    # label is a nrrd image that incorporates the window size and hence, gives a ROI for the GLSZM calculation.
    # The minimum size is set by the user and the mavimum size is the size of the image
    settings = {}
    settings['label'] = 255
    settings['binWidth'] = 1

    # Set-up logging for all the calculations
    log_file = 'D:\\Ankita PhD\\pyradiomics\\log_messages\\log{}.txt'.format(num_img)
    handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
    formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
    handler.setFormatter(formatter)
    radiomics.logger.addHandler(handler)

    # Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
    # Verbositiy level, the logger level will also determine the amount of information printed to the output
    radiomics.logger.setLevel(logging.DEBUG)

    # the size of the mask (same as label) should be the same as that of the image
    mask_x = image_1.GetWidth()
    mask_y = image_1.GetWidth()
    mask_z = image_1.GetWidth()

    print("Initializing window calculcations for image",str(num_img))
    
    # here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
    # sizes decreases with every iteration
    u_window_size = [50, 60, 90, 100, 150, 200, 250, 275]
    global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
    
    for window_size in u_window_size:  
       
        print("Starting the window calculcation for image",str(num_img),"for window size = ", str(window_size)) 
        total_iter = 0

        v1 = []    
        v2 = []
        v3 = []
        v4 = []
        v5 = []
        v6 = []
        v7 = []
        v8 = []
        v9 = []
        v10 = []
        v11 = []
        v12 = []
        v13 = []
        v14 = []
        v15 = []
        v16 = []

        step_x = window_size-1
        step_y = window_size-1
        step_z = window_size-1

        i_loc=0
        j_loc=0
        z_loc=0
        i_loc_max = window_size
        j_loc_max = window_size
        z_loc_max = window_size
        
        
        # x-direction
        while i_loc_max <= mask_x and total_iter < 1000:
            
            #y-direction
            j_loc_max = window_size
            while j_loc_max <= mask_y and total_iter < 1000:
                
                #z-direction
                z_loc_max = window_size
                while z_loc_max <= mask_z and total_iter < 1000:
                    
                    full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
                    full_mask[i_loc:i_loc_max,j_loc:j_loc_max,z_loc:z_loc_max] = 255
                    
                    label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
                    del full_mask

                    bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
                    if correctedMask is not None:
                        label_1 = correctedMask
                    croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
                    del correctedMask, label_1

                    #calculate secondorder stats
                    glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
                    glszmFeatures.enableAllFeatures()
                    print('Calculating GLSZM features...',)
                    result = glszmFeatures.execute()
                    print('done')

                    print('Calculated GLSZM features: ')
                    for (key, val) in six.iteritems(result):
                        if key =='GrayLevelNonUniformity':
                            v1.append(val)
                        if key =='GrayLevelNonUniformityNormalized':
                            v2.append(val)
                        if key =='GrayLevelVariance':
                            v3.append(val)
                        if key =='HighGrayLevelZoneEmphasis':
                            v4.append(val)
                        if key == 'LargeAreaEmphasis':
                            v5.append(val)
                        if key == 'LargeAreaHighGrayLevelEmphasis':
                            v6.append(val)
                        if key == 'LargeAreaLowGrayLevelEmphasis':
                            v7.append(val)
                        if key == 'LowGrayLevelZoneEmphasis':
                            v8.append(val) 
                        if key == 'SizeZoneNonUniformity':
                            v9.append(val)
                        if key == 'SizeZoneNonUniformityNormalized':
                            v10.append(val)
                        if key == 'SmallAreaEmphasis':
                            v11.append(val)
                        if key == 'SmallAreaHighGrayLevelEmphasis':
                            v12.append(val)
                        if key == 'SmallAreaLowGrayLevelEmphasis':
                            v13.append(val)
                        if key == 'ZoneEntropy':
                            v14.append(val)
                        if key == 'ZonePercentage':
                            v15.append(val)
                        if key == 'ZoneVariance':
                            v16.append(val)
                    del key, val, result, glszmFeatures

                    z_loc = z_loc + step_z
                    total_iter = total_iter + 1
                    print(total_iter)
                    z_loc_max = z_loc+window_size

                z_loc = 0
                j_loc = j_loc + step_y
                j_loc_max = j_loc+window_size

            j_loc = 0
            i_loc = i_loc + step_x
            i_loc_max = i_loc+window_size
        
        print("Completed Calculations for Window ", str(window_size), ". The number of iterations is ", str(total_iter)) 
        
        w1 = [window_size] * total_iter
        # Create a data-frame to store all the GLSZM features for the image        
        my_df = pd.DataFrame(list(zip(w1, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16)))
        my_df = my_df.rename(index = str, columns={0: "Window Sizes", 1: "GrayLevelNonUniformity", 2: "GrayLevelNonUniformityNormalized", \
                                   3: "GrayLevelVariance", 4: "HighGrayLevelZoneEmphasis", 5: "LargeAreaEmphasis", \
                                   6: "LargeAreaHighGrayLevelEmphasis", 7: "LargeAreaLowGrayLevelEmphasis", 8: "LowGrayLevelZoneEmphasis", \
                                   9: "SizeZoneNonUniformity", 10: "SizeZoneNonUniformityNormalized", 11: "SmallAreaEmphasis", \
                                   12: "SmallAreaHighGrayLevelEmphasis", 13: "SmallAreaLowGrayLevelEmphasis", 14: "ZoneEntropy", \
                                   15: "ZonePercentage", 16: "ZoneVariance"})    
        
        name_csv = 'glszm_img_{}_w{}.csv'.format(num_img, window_size)
        my_df.to_csv(name_csv, index=False, header=True)
        print("Saved glcm values of window size ", str(window_size))
        del my_df, name_csv 

Starting the automation loop for GLSZM feature extraction
Initializing window calculcations for image 6
Starting the window calculcation for image 6 for window size =  50
Calculating GLSZM features...
done
Calculated GLSZM features: 
1
Calculating GLSZM features...
done
Calculated GLSZM features: 
2
Calculating GLSZM features...
done
Calculated GLSZM features: 
3
Calculating GLSZM features...
done
Calculated GLSZM features: 
4
Calculating GLSZM features...
done
Calculated GLSZM features: 
5
Calculating GLSZM features...
done
Calculated GLSZM features: 
6
Calculating GLSZM features...
done
Calculated GLSZM features: 
7
Calculating GLSZM features...
done
Calculated GLSZM features: 
8
Calculating GLSZM features...
done
Calculated GLSZM features: 
9
Calculating GLSZM features...
done
Calculated GLSZM features: 
10
Calculating GLSZM features...
done
Calculated GLSZM features: 
11
Calculating GLSZM features...
done
Calculated GLSZM features: 
12
Calculating GLSZM features...
done
Calculated 

Calculating GLSZM features...
done
Calculated GLSZM features: 
123
Calculating GLSZM features...
done
Calculated GLSZM features: 
124
Calculating GLSZM features...
done
Calculated GLSZM features: 
125
Calculating GLSZM features...
done
Calculated GLSZM features: 
126
Calculating GLSZM features...
done
Calculated GLSZM features: 
127
Calculating GLSZM features...
done
Calculated GLSZM features: 
128
Calculating GLSZM features...
done
Calculated GLSZM features: 
129
Calculating GLSZM features...
done
Calculated GLSZM features: 
130
Calculating GLSZM features...
done
Calculated GLSZM features: 
131
Calculating GLSZM features...
done
Calculated GLSZM features: 
132
Calculating GLSZM features...
done
Calculated GLSZM features: 
133
Calculating GLSZM features...
done
Calculated GLSZM features: 
134
Calculating GLSZM features...
done
Calculated GLSZM features: 
135
Calculating GLSZM features...
done
Calculated GLSZM features: 
136
Calculating GLSZM features...
done
Calculated GLSZM features: 

Calculating GLSZM features...
done
Calculated GLSZM features: 
246
Calculating GLSZM features...
done
Calculated GLSZM features: 
247
Calculating GLSZM features...
done
Calculated GLSZM features: 
248
Calculating GLSZM features...
done
Calculated GLSZM features: 
249
Calculating GLSZM features...
done
Calculated GLSZM features: 
250
Calculating GLSZM features...
done
Calculated GLSZM features: 
251
Calculating GLSZM features...
done
Calculated GLSZM features: 
252
Calculating GLSZM features...
done
Calculated GLSZM features: 
253
Calculating GLSZM features...
done
Calculated GLSZM features: 
254
Calculating GLSZM features...
done
Calculated GLSZM features: 
255
Calculating GLSZM features...
done
Calculated GLSZM features: 
256
Calculating GLSZM features...
done
Calculated GLSZM features: 
257
Calculating GLSZM features...
done
Calculated GLSZM features: 
258
Calculating GLSZM features...
done
Calculated GLSZM features: 
259
Calculating GLSZM features...
done
Calculated GLSZM features: 

Calculating GLSZM features...
done
Calculated GLSZM features: 
369
Calculating GLSZM features...
done
Calculated GLSZM features: 
370
Calculating GLSZM features...
done
Calculated GLSZM features: 
371
Calculating GLSZM features...
done
Calculated GLSZM features: 
372
Calculating GLSZM features...
done
Calculated GLSZM features: 
373
Calculating GLSZM features...
done
Calculated GLSZM features: 
374
Calculating GLSZM features...
done
Calculated GLSZM features: 
375
Calculating GLSZM features...
done
Calculated GLSZM features: 
376
Calculating GLSZM features...
done
Calculated GLSZM features: 
377
Calculating GLSZM features...
done
Calculated GLSZM features: 
378
Calculating GLSZM features...
done
Calculated GLSZM features: 
379
Calculating GLSZM features...
done
Calculated GLSZM features: 
380
Calculating GLSZM features...
done
Calculated GLSZM features: 
381
Calculating GLSZM features...
done
Calculated GLSZM features: 
382
Calculating GLSZM features...
done
Calculated GLSZM features: 

Calculating GLSZM features...
done
Calculated GLSZM features: 
492
Calculating GLSZM features...
done
Calculated GLSZM features: 
493
Calculating GLSZM features...
done
Calculated GLSZM features: 
494
Calculating GLSZM features...
done
Calculated GLSZM features: 
495
Calculating GLSZM features...
done
Calculated GLSZM features: 
496
Calculating GLSZM features...
done
Calculated GLSZM features: 
497
Calculating GLSZM features...
done
Calculated GLSZM features: 
498
Calculating GLSZM features...
done
Calculated GLSZM features: 
499
Calculating GLSZM features...
done
Calculated GLSZM features: 
500
Calculating GLSZM features...
done
Calculated GLSZM features: 
501
Calculating GLSZM features...
done
Calculated GLSZM features: 
502
Calculating GLSZM features...
done
Calculated GLSZM features: 
503
Calculating GLSZM features...
done
Calculated GLSZM features: 
504
Calculating GLSZM features...
done
Calculated GLSZM features: 
505
Calculating GLSZM features...
done
Calculated GLSZM features: 

...

Calculating GLSZM features...
done
Calculated GLSZM features: 
598
Calculating GLSZM features...
done
Calculated GLSZM features: 
599
Calculating GLSZM features...
done
Calculated GLSZM features: 
600
Calculating GLSZM features...
done
Calculated GLSZM features: 
601
Calculating GLSZM features...
done
Calculated GLSZM features: 
602
Calculating GLSZM features...
done
Calculated GLSZM features: 
603
Calculating GLSZM features...
done
Calculated GLSZM features: 
604
Calculating GLSZM features...
done
Calculated GLSZM features: 
605
Calculating GLSZM features...
done
Calculated GLSZM features: 
606
Calculating GLSZM features...
done
Calculated GLSZM features: 
607
Calculating GLSZM features...
done
Calculated GLSZM features: 
608
Calculating GLSZM features...
done
Calculated GLSZM features: 
609
Calculating GLSZM features...
done
Calculated GLSZM features: 
610
Calculating GLSZM features...
done
Calculated GLSZM features: 
611
Calculating GLSZM features...
done
Calculated GLSZM features: 

Calculating GLSZM features...
done
Calculated GLSZM features: 
721
Calculating GLSZM features...
done
Calculated GLSZM features: 
722
Calculating GLSZM features...
done
Calculated GLSZM features: 
723
Calculating GLSZM features...
done
Calculated GLSZM features: 
724
Calculating GLSZM features...
done
Calculated GLSZM features: 
725
Calculating GLSZM features...
done
Calculated GLSZM features: 
726
Calculating GLSZM features...
done
Calculated GLSZM features: 
727
Calculating GLSZM features...
done
Calculated GLSZM features: 
728
Calculating GLSZM features...
done
Calculated GLSZM features: 
729
Completed Calculations for Window  60 . The number of iterations is  729
Saved glcm values of window size  60
Starting the window calculcation for image 6 for window size =  90
Calculating GLSZM features...
done
Calculated GLSZM features: 
1
Calculating GLSZM features...
done
Calculated GLSZM features: 
2
Calculating GLSZM features...
done
Calculated GLSZM features: 
3
Calculating GLSZM feature

done
Calculated GLSZM features: 
113
Calculating GLSZM features...
done
Calculated GLSZM features: 
114
Calculating GLSZM features...
done
Calculated GLSZM features: 
115
Calculating GLSZM features...
done
Calculated GLSZM features: 
116
Calculating GLSZM features...
done
Calculated GLSZM features: 
117
Calculating GLSZM features...
done
Calculated GLSZM features: 
118
Calculating GLSZM features...
done
Calculated GLSZM features: 
119
Calculating GLSZM features...
done
Calculated GLSZM features: 
120
Calculating GLSZM features...
done
Calculated GLSZM features: 
121
Calculating GLSZM features...
done
Calculated GLSZM features: 
122
Calculating GLSZM features...
done
Calculated GLSZM features: 
123
Calculating GLSZM features...
done
Calculated GLSZM features: 
124
Calculating GLSZM features...
done
Calculated GLSZM features: 
125
Calculating GLSZM features...
done
Calculated GLSZM features: 
126
Calculating GLSZM features...
done
Calculated GLSZM features: 
127
Calculating GLSZM feature

done
Calculated GLSZM features: 
17
Calculating GLSZM features...
done
Calculated GLSZM features: 
18
Calculating GLSZM features...
done
Calculated GLSZM features: 
19
Calculating GLSZM features...
done
Calculated GLSZM features: 
20
Calculating GLSZM features...
done
Calculated GLSZM features: 
21
Calculating GLSZM features...
done
Calculated GLSZM features: 
22
Calculating GLSZM features...
done
Calculated GLSZM features: 
23
Calculating GLSZM features...
done
Calculated GLSZM features: 
24
Calculating GLSZM features...
done
Calculated GLSZM features: 
25
Calculating GLSZM features...
done
Calculated GLSZM features: 
26
Calculating GLSZM features...
done
Calculated GLSZM features: 
27
Calculating GLSZM features...
done
Calculated GLSZM features: 
28
Calculating GLSZM features...
done
Calculated GLSZM features: 
29
Calculating GLSZM features...
done
Calculated GLSZM features: 
30
Calculating GLSZM features...
done
Calculated GLSZM features: 
31
Calculating GLSZM features...
done
Calcu

Calculating GLSZM features...
done
Calculated GLSZM features: 
14
Calculating GLSZM features...
done
Calculated GLSZM features: 
15
Calculating GLSZM features...
done
Calculated GLSZM features: 
16
Calculating GLSZM features...
done
Calculated GLSZM features: 
17
Calculating GLSZM features...
done
Calculated GLSZM features: 
18
Calculating GLSZM features...
done
Calculated GLSZM features: 
19
Calculating GLSZM features...
done
Calculated GLSZM features: 
20
Calculating GLSZM features...
done
Calculated GLSZM features: 
21
Calculating GLSZM features...
done
Calculated GLSZM features: 
22
Calculating GLSZM features...
done
Calculated GLSZM features: 
23
Calculating GLSZM features...
done
Calculated GLSZM features: 
24
Calculating GLSZM features...
done
Calculated GLSZM features: 
25
Calculating GLSZM features...
done
Calculated GLSZM features: 
26
Calculating GLSZM features...
done
Calculated GLSZM features: 
27
Completed Calculations for Window  150 . The number of iterations is  27
Sav

#### MJ Carbonate: Automate the GLSZM feature extraction process - Window Based - for more than half the image size

In [20]:
%%time
print("Starting the automation loop for GLSZM feature extraction")

# All the greyscale images of various porous media types are named as data#.nrrd. 
# Iterate over the names to calculate the GLSZM features
for num_img in range(6, 7):
    
    # Create the variables for image and label
    global image_1, label_1

    # Path of the images used in REV analysis
    path ='D:\\Ankita PhD\\pyradiomics\\data\\data{}.nrrd'.format(num_img)
    
    # Read the nrrd image using the SITK library. Nrrd images are 8-bit images with intensity values between 0 to 255
    image_1 = sitk.ReadImage(path) 
    
    # Set the settings paramter needed for the bin_width and label
    # Bin_Width = 1 (as I don't want any further reduction in greyscale values and want to preserve the information)
    # label is a nrrd image that incorporates the window size and hence, gives a ROI for the GLSZM calculation.
    # The minimum size is set by the user and the mavimum size is the size of the image
    settings = {}
    settings['label'] = 255
    settings['binWidth'] = 1

    # Set-up logging for all the calculations
    log_file = 'D:\\Ankita PhD\\pyradiomics\\log_messages\\log{}.txt'.format(num_img)
    handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
    formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
    handler.setFormatter(formatter)
    radiomics.logger.addHandler(handler)

    # Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
    # Verbositiy level, the logger level will also determine the amount of information printed to the output
    radiomics.logger.setLevel(logging.DEBUG)

    # the size of the mask (same as label) should be the same as that of the image
    mask_x = image_1.GetWidth()
    mask_y = image_1.GetWidth()
    mask_z = image_1.GetWidth()

    print("Initializing window calculcations for image",str(num_img))
    
    # here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
    # sizes decreases with every iteration
    u_window_size = [300, 350, 400, 450, 500, 525]
    global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
    
    for window_size in u_window_size:  
       
        print("Starting the window calculcation for image",str(num_img),"for window size = ", str(window_size)) 
        total_iter = 0
        v1 = []
        v2 = []
        v3 = []
        v4 = []
        v5 = []
        v6 = []
        v7 = []
        v8 = []
        v9 = []
        v10 = []
        v11 = []
        v12 = []
        v13 = []
        v14 = []
        v15 = []
        v16 = []

        step_x = window_size-1
        step_y = window_size-1
        step_z = window_size-1

        i_loc=0
        j_loc=0
        z_loc=0
        i_loc_max = window_size
        j_loc_max = window_size
        z_loc_max = window_size

        # Position 1
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc_max,j_loc:j_loc_max,z_loc:z_loc_max] = 255

        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        #calculate secondorder stats
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        print('Calculating GLSZM features...',)
        result = glszmFeatures.execute()
        print('done')

        print('Calculated GLSZM features: ')
        for (key, val) in six.iteritems(result):
            if key =='GrayLevelNonUniformity':
                v1.append(val)
            if key =='GrayLevelNonUniformityNormalized':
                v2.append(val)
            if key =='GrayLevelVariance':
                v3.append(val)
            if key =='HighGrayLevelZoneEmphasis':
                v4.append(val)
            if key == 'LargeAreaEmphasis':
                v5.append(val)
            if key == 'LargeAreaHighGrayLevelEmphasis':
                v6.append(val)
            if key == 'LargeAreaLowGrayLevelEmphasis':
                v7.append(val)
            if key == 'LowGrayLevelZoneEmphasis':
                v8.append(val) 
            if key == 'SizeZoneNonUniformity':
                v9.append(val)
            if key == 'SizeZoneNonUniformityNormalized':
                v10.append(val)
            if key == 'SmallAreaEmphasis':
                v11.append(val)
            if key == 'SmallAreaHighGrayLevelEmphasis':
                v12.append(val)
            if key == 'SmallAreaLowGrayLevelEmphasis':
                v13.append(val)
            if key == 'ZoneEntropy':
                v14.append(val)
            if key == 'ZonePercentage':
                v15.append(val)
            if key == 'ZoneVariance':
                v16.append(val)
        del key, val, result, glszmFeatures
        
        #Position 2
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[mask_x-window_size+1:mask_x, mask_y-window_size+1:mask_y, mask_z-window_size+1:mask_z] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        #calculate secondorder stats
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        print('Calculating GLSZM features...',)
        result = glszmFeatures.execute()
        print('done')

        print('Calculated GLSZM features: ')
        for (key, val) in six.iteritems(result):
            if key =='GrayLevelNonUniformity':
                v1.append(val)
            if key =='GrayLevelNonUniformityNormalized':
                v2.append(val)
            if key =='GrayLevelVariance':
                v3.append(val)
            if key =='HighGrayLevelZoneEmphasis':
                v4.append(val)
            if key == 'LargeAreaEmphasis':
                v5.append(val)
            if key == 'LargeAreaHighGrayLevelEmphasis':
                v6.append(val)
            if key == 'LargeAreaLowGrayLevelEmphasis':
                v7.append(val)
            if key == 'LowGrayLevelZoneEmphasis':
                v8.append(val) 
            if key == 'SizeZoneNonUniformity':
                v9.append(val)
            if key == 'SizeZoneNonUniformityNormalized':
                v10.append(val)
            if key == 'SmallAreaEmphasis':
                v11.append(val)
            if key == 'SmallAreaHighGrayLevelEmphasis':
                v12.append(val)
            if key == 'SmallAreaLowGrayLevelEmphasis':
                v13.append(val)
            if key == 'ZoneEntropy':
                v14.append(val)
            if key == 'ZonePercentage':
                v15.append(val)
            if key == 'ZoneVariance':
                v16.append(val)
        del key, val, result, glszmFeatures
        
        #Position 3
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc_max, mask_y-window_size+1:mask_y, z_loc:z_loc_max] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        #calculate secondorder stats
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        print('Calculating GLSZM features...',)
        result = glszmFeatures.execute()
        print('done')

        print('Calculated GLSZM features: ')
        for (key, val) in six.iteritems(result):
            if key =='GrayLevelNonUniformity':
                v1.append(val)
            if key =='GrayLevelNonUniformityNormalized':
                v2.append(val)
            if key =='GrayLevelVariance':
                v3.append(val)
            if key =='HighGrayLevelZoneEmphasis':
                v4.append(val)
            if key == 'LargeAreaEmphasis':
                v5.append(val)
            if key == 'LargeAreaHighGrayLevelEmphasis':
                v6.append(val)
            if key == 'LargeAreaLowGrayLevelEmphasis':
                v7.append(val)
            if key == 'LowGrayLevelZoneEmphasis':
                v8.append(val) 
            if key == 'SizeZoneNonUniformity':
                v9.append(val)
            if key == 'SizeZoneNonUniformityNormalized':
                v10.append(val)
            if key == 'SmallAreaEmphasis':
                v11.append(val)
            if key == 'SmallAreaHighGrayLevelEmphasis':
                v12.append(val)
            if key == 'SmallAreaLowGrayLevelEmphasis':
                v13.append(val)
            if key == 'ZoneEntropy':
                v14.append(val)
            if key == 'ZonePercentage':
                v15.append(val)
            if key == 'ZoneVariance':
                v16.append(val)
        del key, val, result, glszmFeatures
        
        #Position 4
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[mask_x-window_size+1:mask_x, mask_y-window_size+1:mask_y, z_loc:z_loc_max] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        #calculate secondorder stats
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        print('Calculating GLSZM features...',)
        result = glszmFeatures.execute()
        print('done')

        print('Calculated GLSZM features: ')
        for (key, val) in six.iteritems(result):
            if key =='GrayLevelNonUniformity':
                v1.append(val)
            if key =='GrayLevelNonUniformityNormalized':
                v2.append(val)
            if key =='GrayLevelVariance':
                v3.append(val)
            if key =='HighGrayLevelZoneEmphasis':
                v4.append(val)
            if key == 'LargeAreaEmphasis':
                v5.append(val)
            if key == 'LargeAreaHighGrayLevelEmphasis':
                v6.append(val)
            if key == 'LargeAreaLowGrayLevelEmphasis':
                v7.append(val)
            if key == 'LowGrayLevelZoneEmphasis':
                v8.append(val) 
            if key == 'SizeZoneNonUniformity':
                v9.append(val)
            if key == 'SizeZoneNonUniformityNormalized':
                v10.append(val)
            if key == 'SmallAreaEmphasis':
                v11.append(val)
            if key == 'SmallAreaHighGrayLevelEmphasis':
                v12.append(val)
            if key == 'SmallAreaLowGrayLevelEmphasis':
                v13.append(val)
            if key == 'ZoneEntropy':
                v14.append(val)
            if key == 'ZonePercentage':
                v15.append(val)
            if key == 'ZoneVariance':
                v16.append(val)
        del key, val, result, glszmFeatures
        
        #Position 5
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc_max, j_loc:j_loc_max, mask_z-window_size+1:mask_z] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        #calculate secondorder stats
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        print('Calculating GLSZM features...',)
        result = glszmFeatures.execute()
        print('done')

        print('Calculated GLSZM features: ')
        for (key, val) in six.iteritems(result):
            if key =='GrayLevelNonUniformity':
                v1.append(val)
            if key =='GrayLevelNonUniformityNormalized':
                v2.append(val)
            if key =='GrayLevelVariance':
                v3.append(val)
            if key =='HighGrayLevelZoneEmphasis':
                v4.append(val)
            if key == 'LargeAreaEmphasis':
                v5.append(val)
            if key == 'LargeAreaHighGrayLevelEmphasis':
                v6.append(val)
            if key == 'LargeAreaLowGrayLevelEmphasis':
                v7.append(val)
            if key == 'LowGrayLevelZoneEmphasis':
                v8.append(val) 
            if key == 'SizeZoneNonUniformity':
                v9.append(val)
            if key == 'SizeZoneNonUniformityNormalized':
                v10.append(val)
            if key == 'SmallAreaEmphasis':
                v11.append(val)
            if key == 'SmallAreaHighGrayLevelEmphasis':
                v12.append(val)
            if key == 'SmallAreaLowGrayLevelEmphasis':
                v13.append(val)
            if key == 'ZoneEntropy':
                v14.append(val)
            if key == 'ZonePercentage':
                v15.append(val)
            if key == 'ZoneVariance':
                v16.append(val)
        del key, val, result, glszmFeatures
        
        #Position 6
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[mask_x-window_size+1:mask_x, j_loc:j_loc_max, mask_z-window_size+1:mask_z] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        #calculate secondorder stats
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        print('Calculating GLSZM features...',)
        result = glszmFeatures.execute()
        print('done')

        print('Calculated GLSZM features: ')
        for (key, val) in six.iteritems(result):
            if key =='GrayLevelNonUniformity':
                v1.append(val)
            if key =='GrayLevelNonUniformityNormalized':
                v2.append(val)
            if key =='GrayLevelVariance':
                v3.append(val)
            if key =='HighGrayLevelZoneEmphasis':
                v4.append(val)
            if key == 'LargeAreaEmphasis':
                v5.append(val)
            if key == 'LargeAreaHighGrayLevelEmphasis':
                v6.append(val)
            if key == 'LargeAreaLowGrayLevelEmphasis':
                v7.append(val)
            if key == 'LowGrayLevelZoneEmphasis':
                v8.append(val) 
            if key == 'SizeZoneNonUniformity':
                v9.append(val)
            if key == 'SizeZoneNonUniformityNormalized':
                v10.append(val)
            if key == 'SmallAreaEmphasis':
                v11.append(val)
            if key == 'SmallAreaHighGrayLevelEmphasis':
                v12.append(val)
            if key == 'SmallAreaLowGrayLevelEmphasis':
                v13.append(val)
            if key == 'ZoneEntropy':
                v14.append(val)
            if key == 'ZonePercentage':
                v15.append(val)
            if key == 'ZoneVariance':
                v16.append(val)
        del key, val, result, glszmFeatures
        
        #Position 7
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc_max, mask_y-window_size+1:mask_y, mask_z-window_size+1:mask_z] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        #calculate secondorder stats
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        print('Calculating GLSZM features...',)
        result = glszmFeatures.execute()
        print('done')

        print('Calculated GLSZM features: ')
        for (key, val) in six.iteritems(result):
            if key =='GrayLevelNonUniformity':
                v1.append(val)
            if key =='GrayLevelNonUniformityNormalized':
                v2.append(val)
            if key =='GrayLevelVariance':
                v3.append(val)
            if key =='HighGrayLevelZoneEmphasis':
                v4.append(val)
            if key == 'LargeAreaEmphasis':
                v5.append(val)
            if key == 'LargeAreaHighGrayLevelEmphasis':
                v6.append(val)
            if key == 'LargeAreaLowGrayLevelEmphasis':
                v7.append(val)
            if key == 'LowGrayLevelZoneEmphasis':
                v8.append(val) 
            if key == 'SizeZoneNonUniformity':
                v9.append(val)
            if key == 'SizeZoneNonUniformityNormalized':
                v10.append(val)
            if key == 'SmallAreaEmphasis':
                v11.append(val)
            if key == 'SmallAreaHighGrayLevelEmphasis':
                v12.append(val)
            if key == 'SmallAreaLowGrayLevelEmphasis':
                v13.append(val)
            if key == 'ZoneEntropy':
                v14.append(val)
            if key == 'ZonePercentage':
                v15.append(val)
            if key == 'ZoneVariance':
                v16.append(val)
        del key, val, result, glszmFeatures
        
        #Position 8
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[mask_x-window_size+1:mask_x, j_loc:j_loc_max, z_loc:z_loc_max] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        #calculate secondorder stats
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        print('Calculating GLSZM features...',)
        result = glszmFeatures.execute()
        print('done')

        print('Calculated GLSZM features: ')
        for (key, val) in six.iteritems(result):
            if key =='GrayLevelNonUniformity':
                v1.append(val)
            if key =='GrayLevelNonUniformityNormalized':
                v2.append(val)
            if key =='GrayLevelVariance':
                v3.append(val)
            if key =='HighGrayLevelZoneEmphasis':
                v4.append(val)
            if key == 'LargeAreaEmphasis':
                v5.append(val)
            if key == 'LargeAreaHighGrayLevelEmphasis':
                v6.append(val)
            if key == 'LargeAreaLowGrayLevelEmphasis':
                v7.append(val)
            if key == 'LowGrayLevelZoneEmphasis':
                v8.append(val) 
            if key == 'SizeZoneNonUniformity':
                v9.append(val)
            if key == 'SizeZoneNonUniformityNormalized':
                v10.append(val)
            if key == 'SmallAreaEmphasis':
                v11.append(val)
            if key == 'SmallAreaHighGrayLevelEmphasis':
                v12.append(val)
            if key == 'SmallAreaLowGrayLevelEmphasis':
                v13.append(val)
            if key == 'ZoneEntropy':
                v14.append(val)
            if key == 'ZonePercentage':
                v15.append(val)
            if key == 'ZoneVariance':
                v16.append(val)
        del key, val, result, glszmFeatures
        
        print("Completed Calculations for Window ", str(window_size), ". The number of iterations is ", str(total_iter)) 
        
        w1 = [window_size] * total_iter
        # Create a data-frame to store all the GLSZM features for the image        
        my_df = pd.DataFrame(list(zip(w1, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16)))
        my_df = my_df.rename(index = str, columns={0: "Window Sizes", 1: "GrayLevelNonUniformity", 2: "GrayLevelNonUniformityNormalized", \
                                   3: "GrayLevelVariance", 4: "HighGrayLevelZoneEmphasis", 5: "LargeAreaEmphasis", \
                                   6: "LargeAreaHighGrayLevelEmphasis", 7: "LargeAreaLowGrayLevelEmphasis", 8: "LowGrayLevelZoneEmphasis", \
                                   9: "SizeZoneNonUniformity", 10: "SizeZoneNonUniformityNormalized", 11: "SmallAreaEmphasis", \
                                   12: "SmallAreaHighGrayLevelEmphasis", 13: "SmallAreaLowGrayLevelEmphasis", 14: "ZoneEntropy", \
                                   15: "ZonePercentage", 16: "ZoneVariance"})    
        
        name_csv = 'glszm_img_{}_w{}.csv'.format(num_img, window_size)
        my_df.to_csv(name_csv, index=False, header=True)
        print("Saved glcm values of window size ", str(window_size))
        del my_df, name_csv 

Starting the automation loop for GLSZM feature extraction
Initializing window calculcations for image 6
Starting the window calculcation for image 6 for window size =  300
Calculating GLSZM features...
done
Calculated GLSZM features: 
Calculating GLSZM features...
done
Calculated GLSZM features: 
Calculating GLSZM features...
done
Calculated GLSZM features: 
Calculating GLSZM features...
done
Calculated GLSZM features: 
Calculating GLSZM features...
done
Calculated GLSZM features: 
Calculating GLSZM features...
done
Calculated GLSZM features: 
Calculating GLSZM features...
done
Calculated GLSZM features: 
Calculating GLSZM features...
done
Calculated GLSZM features: 
Completed Calculations for Window  300 . The number of iterations is  8
Saved glcm values of window size  300
Starting the window calculcation for image 6 for window size =  350
Calculating GLSZM features...
done
Calculated GLSZM features: 
Calculating GLSZM features...
done
Calculated GLSZM features: 
Calculating GLSZM fe

In [9]:
x = np.array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],
       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],
       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [10]:
x

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [11]:
x[0:2,0:2,0:2]

array([[[ 0,  1],
        [ 3,  4]],

       [[ 9, 10],
        [12, 13]]])